In [6]:
# # If data set is too large, use the random sample below!

# import pandas as pd
# import random

# # since the file has over 2 million rows, we'll take a sample of 10000
# filename = "./lending-club-loan-data/loan.csv"
# n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
# s = 10000 #desired sample size
# skip = sorted(random.sample(xrange(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
# df = pd.read_csv(filename, skiprows=skip)


In [9]:
import pandas as pd
filename = "./lending-club-loan-data/loan.csv"
loans = pd.read_csv(filename, low_memory=False)

In [10]:
# Take a look at how big the data set is...
# looks like it has over 2 million rows!
loans.shape

(2260668, 145)

In [12]:
# Look at the data type of each column...
# well there are a lot of features, with floats, strings, and ints
loans.dtypes

id                                            float64
member_id                                     float64
loan_amnt                                       int64
funded_amnt                                     int64
funded_amnt_inv                               float64
term                                           object
int_rate                                      float64
installment                                   float64
grade                                          object
sub_grade                                      object
emp_title                                      object
emp_length                                     object
home_ownership                                 object
annual_inc                                    float64
verification_status                            object
issue_d                                        object
loan_status                                    object
pymnt_plan                                     object
url                         

In [25]:
# Checking for null values...
# there are quite a few columns with all nulls and some with mostly nulls
# this is good because we have over 100 features and reducing our features will help!
loans_null = loans.isnull().sum()
loans_null

id                                            2260668
member_id                                     2260668
loan_amnt                                           0
funded_amnt                                         0
funded_amnt_inv                                     0
term                                                0
int_rate                                            0
installment                                         0
grade                                               0
sub_grade                                           0
emp_title                                      166969
emp_length                                     146907
home_ownership                                      0
annual_inc                                          4
verification_status                                 0
issue_d                                             0
loan_status                                         0
pymnt_plan                                          0
url                         

In [43]:
# Lets get the columns that only have up to 10% of the values as null
# filtering the columns has now reduced the number of features for 145 to 86!
new_features = loans_null[loans_null<250000].index.tolist()
print(len(new_features))

86


In [44]:
# Even when filtering for no null values, we still have 32 feaures
# 32 is a lot so let's go with that for now
new_features = loans_null[loans_null<1].index.tolist()
print(len(new_features))
print(new_features)

32
['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'addr_state', 'revol_bal', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'policy_code', 'application_type', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag']


In [36]:
loans = loans[new_features]

In [53]:
# Exploring potential y value to predict
#  this will help determine which future should be our predicted value (y)

print("#### loan_status ####")
print(loans.loan_status.value_counts())

#### loan_status ####
Fully Paid                                             1041952
Current                                                 919695
Charged Off                                             261655
Late (31-120 days)                                       21897
In Grace Period                                           8952
Late (16-30 days)                                         3737
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     31
Name: loan_status, dtype: int64


In [51]:
# Looks like we can use loan_status as the value to predict , y
# We'll want to simplify it slightly by only using Fully Paid and Charged Off to train and test our model
# Reducing loans data set one more time...
final_loans = loans.loc[(loans['loan_status'] == 'Fully Paid') | (loans['loan_status'] == 'Charged Off')]
final_loans['loan_status'].value_counts()

# At this point, for the final project we'll be categorizating loans into whether they are fully paid off or charged off.

Fully Paid     1041952
Charged Off     261655
Name: loan_status, dtype: int64

In [54]:
# Using value counts to see what kind of values are available in some of the features
# We're seeing some features with only 1 values. Looks like we can further reduce our features!
print("\n#### debt_settlement_flag ####")
print(final_loans.debt_settlement_flag.value_counts())

print("\n#### verification_status ####")
print(final_loans.verification_status.value_counts())

print("\n#### pymnt_plan ####")
print(final_loans.pymnt_plan.value_counts())

print("\n#### initial_list_status ####")
print(final_loans.initial_list_status.value_counts())

print("\n#### policy_code ####")
print(final_loans.policy_code.value_counts())

print("\n#### application_type ####")
print(final_loans.application_type.value_counts())

print("\n#### hardship_flag ####")
print(final_loans.hardship_flag.value_counts())


#### debt_settlement_flag ####
N    1271618
Y      31989
Name: debt_settlement_flag, dtype: int64

#### verification_status ####
Source Verified    503726
Verified           407676
Not Verified       392205
Name: verification_status, dtype: int64

#### pymnt_plan ####
n    1303607
Name: pymnt_plan, dtype: int64

#### initial_list_status ####
w    751214
f    552393
Name: initial_list_status, dtype: int64

#### policy_code ####
1    1303607
Name: policy_code, dtype: int64

#### application_type ####
Individual    1280370
Joint App       23237
Name: application_type, dtype: int64

#### hardship_flag ####
N    1303606
Y          1
Name: hardship_flag, dtype: int64


In [68]:
# get list of features with only 1 type of value
final_loan_columns = final_loans.columns.tolist()
useful_loan_columns = []
for c in final_loan_columns:
    if len(final_loans[c].unique()) > 1:
        useful_loan_columns.append(c)
print(len(useful_loan_columns))
useful_loan_columns

28


['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'purpose',
 'addr_state',
 'revol_bal',
 'initial_list_status',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'application_type',
 'hardship_flag',
 'disbursement_method',
 'debt_settlement_flag']

After some poor df naming and feature exploration, we've our X and y.

X has been reduced from 145 to 27 by excluding the features with null values and
exlcuding features that only have 1 unique value.
Features with only 1 unique value have been excluded because they don't add any value to
the prediction since it's the same constant.

In [69]:
"""
After some poor df naming and feature exploration,
"""
loans = final_loans[useful_loan_columns]
X = loans[['loan_amnt',
             'funded_amnt',
             'funded_amnt_inv',
             'term',
             'int_rate',
             'installment',
             'grade',
             'sub_grade',
             'home_ownership',
             'verification_status',
             'issue_d',
             'purpose',
             'addr_state',
             'revol_bal',
             'initial_list_status',
             'total_pymnt',
             'total_pymnt_inv',
             'total_rec_prncp',
             'total_rec_int',
             'total_rec_late_fee',
             'recoveries',
             'collection_recovery_fee',
             'last_pymnt_amnt',
             'application_type',
             'hardship_flag',
             'disbursement_method',
             'debt_settlement_flag']]
y = loans[['loan_status']]
print(X.head(3))
print(y.head(3))

     loan_amnt  funded_amnt  funded_amnt_inv        term  int_rate  \
100      30000        30000          30000.0   36 months     22.35   
152      40000        40000          40000.0   60 months     16.14   
170      20000        20000          20000.0   36 months      7.56   

     installment grade sub_grade home_ownership verification_status  \
100      1151.16     D        D5       MORTGAGE     Source Verified   
152       975.71     C        C4       MORTGAGE            Verified   
170       622.68     A        A3       MORTGAGE        Not Verified   

             ...          total_rec_prncp total_rec_int total_rec_late_fee  \
100          ...                  30000.0         26.44                0.0   
152          ...                  40000.0        856.68                0.0   
170          ...                  20000.0        215.79                0.0   

     recoveries collection_recovery_fee  last_pymnt_amnt  application_type  \
100         0.0                     0.0    